<a href="https://colab.research.google.com/github/Irenekayla/ML_Notebooks/blob/main/Denoising_and_super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from google.colab import drive
import cv2
import os

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model


In [3]:
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/ICIP training data'

Mounted at /content/drive


In [4]:
input_img = Input(shape=(300, 300, 1))


x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(x)


x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [11]:
def calculate_msr(image):
    mean = np.mean(image)
    std_dev = np.std(image)
    return mean / std_dev

# Function to calculate CNR (Contrast-to-Noise Ratio)
def calculate_cnr(image, background_roi, foreground_roi):
    background_mean = np.mean(image[background_roi])
    background_std = np.std(image[background_roi])
    foreground_mean = np.mean(image[foreground_roi])
    foreground_std = np.std(image[foreground_roi])
    return (foreground_mean - background_mean) / (0.5 * (background_std + foreground_std))


In [21]:
for file in os.listdir(dataset_path):
    if file.endswith('.tiff'):
        # Load the OCT B-scan
        image = cv2.imread(os.path.join(dataset_path, file), cv2.IMREAD_GRAYSCALE)

        # De-noising task
        denoised_image = autoencoder.predict(image.reshape(1, 300, 300, 1))

        # Calculate evaluation metrics for de-noising
        msr = calculate_msr(denoised_image)
        cnr = calculate_cnr(denoised_image, background_roi, foreground_roi)

In [22]:
print(f'File: {file}')
print(f'De-noising MSR: {msr}')
print(f'De-noising CNR: {cnr}')

File: 1


NameError: name 'msr' is not defined